In [18]:
import glob
import pandas as pd
import json
import csv
from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Train'  # use your path
allFiles = glob.glob(path + "/*.csv")


# Sentence Lists
sentence_text = []
sentence_class = []



#action first party
total_data_samples0 = []
total_data_number0 = []
total_data_label0 = []

#Personal Info Type
total_data_samples1 = []
total_data_number1 = []
total_data_label1 = []


#Purpose
total_data_samples2 = []
total_data_number2 = []
total_data_label2 = []


#None
total_data_samples3 = []
total_data_number3 = []
total_data_label3 = []

#checking counts, we can remove them later
count_1 = 0
count_2 = 0
count_3 = 0

#checking counts, we can remove them later
count_11 = 0
count_22 = 0
count_33 = 0

cnt = 0
cnt0 = 0
cnt1 = 0
cnt2 = 0
cnt3 = 0
choice = 0
allFiles = allFiles[:85]


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives
sen_actioFP = []
sen_actioFP_class = []

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives
sen_personalIT = []
sen_personalIT_class = []

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives
sen_purpose = []
sen_purpose_class = []

sen3_pos = 0

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head
#     print(len(list(soup.descendants)))
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
#         print(element)
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
#                 print(len(string))
                start_index.append(count)
#                 print(count)
    doc = '\n'.join(text)
    return doc

sentence_list = []

for file in allFiles:
    dictCollection = {"Action First-Party":[],
                      "Purpose":[],
                      "Personal Information Type":[],
                      "None":[]
                      }

    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    # df_tail = df.tail(1)[4]
    # print(df_tail)
    number_of_segments = len(df) + 1

    file = file.split('Train/', 1)[1]
#     print(file)

    ### beautifulsuop on html doc
    file_html = 'Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
#     print(file_html)
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)
#     print(len(indeces))
#     print(len(ls))
    ###
    
    
    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
#             choice = 1
            parse_json = json.loads(str(df[6][i]))
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                choice = 1
                count_11 += 1
                total_data_samples0.append(parse_json["Action First-Party"]["selectedText"])
                total_data_number0.append(0)
                total_data_label0.append("Action First-Party")
                str_index = parse_json["Action First-Party"]["startIndexInSegment"]
                end_index = parse_json["Action First-Party"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_1 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        sentence_list.append([ls[index],"Action_First-Party_positive","positive"])
                        sentence_text.append(ls[index])
                        sentence_class.append("Action_First-Party_positive")
                        sen0_pos += 1
                        sen1_neg += 1
                        sen2_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("pos")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")
                    else: 
                        sentence_list.append([ls[index],"Action_First-Party_negative","negative"])
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")
                        sentence_text.append(ls[index])
                        sentence_class.append("Action_First-Party_negative")
                cnt0 = cnt0+1
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                choice = 1
                count_22 += 1
                total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                total_data_number1.append(1)
                total_data_label1.append("Personal Information Type")                
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_2 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
                        sentence_text.append(ls[index])
                        sentence_class.append("Personal_Information_Type_positive")
                        sen1_pos += 1
                        sen0_neg += 1
                        sen2_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("pos")                        
                        sen_purpose_class.append("neg")                       
                    else: 
                        sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
                        sentence_text.append(ls[index])
                        sentence_class.append("Personal_Information_Type_negative")
                        sen1_neg += 1
                        sen0_neg += 1
                        sen2_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")                        
                cnt1 = cnt1+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                choice = 1            
                count_33 += 1                
                total_data_samples2.append(parse_json["Purpose"]["selectedText"])
                total_data_number2.append(2)
                total_data_label2.append("Purpose")     
                str_index = parse_json["Purpose"]["startIndexInSegment"]
                end_index = parse_json["Purpose"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_3 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        sentence_list.append([ls[index],"Purpose_positive","positive"])
                        sentence_text.append(ls[index])
                        sentence_class.append("Purpose_positive")
                        sen2_pos += 1
                        sen0_neg += 1
                        sen1_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("pos")                        
                    else: 
                        sentence_list.append([ls[index],"Purpose_negative","negative"])
                        sentence_text.append(ls[index])
                        sentence_class.append("Purpose_negative")
                        sen2_neg += 1
                        sen0_neg += 1
                        sen1_neg += 1
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")                        
                cnt2 = cnt2+1
#         if df[5][i] == "Third Party Sharing/Collection":
#             parse_json = json.loads(str(df[6][i]))
# #             print(parse_json)
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
#                 total_data_number1.append(1)
#                 total_data_label1.append("Personal Information Type")
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose") 
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_3 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sentence_list.append([ls[index],"Purpose_positive","positive"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Purpose_positive")
#                         sen2_pos += 1
#                     else: 
#                         sentence_list.append([ls[index],"Purpose_negative","negative"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Purpose_negative")
#                         sen2_neg += 1
#                 cnt2 = cnt2+1
#         if df[5][i] == "User Choice/Control":
# #             choice = 1
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
# #                 print(parse_json["Personal Information Type"]["startIndexInSegment"],parse_json["Personal Information Type"]["endIndexInSegment"])
#                 total_data_number1.append(1)
#                 total_data_label1.append("Personal Information Type")
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose")
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_3 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sentence_list.append([ls[index],"Purpose_positive","positive"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Purpose_positive")
#                         sen2_pos += 1
#                     else: 
#                         sentence_list.append([ls[index],"Purpose_negative","negative"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Purpose_negative")
#                         sen2_neg += 1
#                 cnt2 = cnt2+1
#         if df[5][i] == "Data Retention":
# #             choice = 1
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
#                 total_data_number1.append(1)
#                 total_data_label1.append("Personal Information Type")
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         sentence_text.append(ls[index])
#                         sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#         if choice==0:
# #             print("practis is -->", df[5][i])
#             parse_json = json.loads(str(df[6][i]))
# #             print(parse_json)
# #             print(len(df[6][i]))
# #             print(parse_json.keys())
#             attributes = parse_json.keys()
# #             print(attributes)
#             for k in attributes:
# #                 print(k)
#                 if parse_json[k]['startIndexInSegment'] != -1:
# #                     print(parse_json[k]['selectedText'])
#                     total_data_samples3.append(parse_json[k]['selectedText'])
#                     total_data_number3.append(3)
#                     total_data_label3.append("None")
#                     str_index = parse_json[k]["startIndexInSegment"]
#                     end_index = parse_json[k]["endIndexInSegment"]
#                     for index in range(len(ls)):
#                         count_3 += 1
#                         if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                            (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                            (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                            (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                             sentence_list.append([ls[index],"None_positive","positive"])
#                             sentence_text.append(ls[index])
#                             sentence_class.append("none")
#                             sen3_pos += 1
                        
#                     cnt3 = cnt3+1

                
# print("----- check counts")                
# print(count_1, count_11)
# print(count_2, count_22)
# print(count_3, count_33)                
                                
print("----- postive and -ve sentences")                
print(sen0_pos,sen0_neg)
print(sen1_pos,sen1_neg)
print(sen2_pos,sen2_neg)
print(len(sentence_list))

print("----- postive and -ve sentence count")
print(len(sen_actioFP),len(sen_actioFP_neg))
print(len(sen_personalIT),len(sen_personalIT_neg))
print(len(sen_purpose),len(sen_purpose_neg))
print(len(sentence_list))


# print("---")
# print(cnt0)
# print(cnt1)
# print(cnt2)
# print(cnt3)
# print("--")
# print(cnt0+cnt1+cnt2+cnt3)
# print(len(total_data_samples0))
# print(len(total_data_samples1))
# print(len(total_data_samples2))
# print(len(total_data_samples3)) # 22536 8780

# print(len(sentence_text))

#942670
#942670

----- check counts
424953 3673
517717 4400
445874 3744
----- postive and -ve sentences
4464 1384080
5301 1383243
5356 1383188
1388544
----- postive and -ve sentence count
1388544 0
1388544 0
1388544 0
1388544
---
3673
4400
3744
0
--
11817
3673
4400
3744
0
1388544


In [19]:
print(sen_actioFP[:10])

['Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'To better inform you of our policy concerning user privacy, we have adopted the following terms.', 'Please note that these terms are subject to change, and any such changes will be included on this page.', '|||Information that Sci-News.com May Collect Online\nSci-News.com may collect and process the following data about you:\n- information that you provide by filling in forms on our site, including names, e-mail and website addresses; we may also ask you for information for other purposes, for example when you report a problem with our site;\n|||- if you contact us, we may keep a record of that correspondence;\n|||- details of your visits to our site including, but not limited to, traffic data, location data, weblogs and other communication data.', '|||Sci-News.com does not knowingly collect or solicit personal information from anyone under the age of 13.', 'We assume that minors 13 years of age o

In [38]:
### Extraacing the test data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Test'  # use your path
allFiles = glob.glob(path + "/*.csv")
print("len of all files", len(allFiles))

# Sentence Lists
test_sentence_text = []
test_sentence_class = []
test_sentence_list = []



#action first party
total_data_samples0 = []
total_data_number0 = []
total_data_label0 = []

#Personal Info Type
total_data_samples1 = []
total_data_number1 = []
total_data_label1 = []


#Purpose
total_data_samples2 = []
total_data_number2 = []
total_data_label2 = []


#None
total_data_samples3 = []
total_data_number3 = []
total_data_label3 = []

#checking counts, we can remove them later
count_1 = 0
count_2 = 0
count_3 = 0

#checking counts, we can remove them later
count_11 = 0
count_22 = 0
count_33 = 0


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives
test_sen_actioFP = []
test_sen_actioFP_class = []

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives
test_sen_personalIT = []
test_sen_personalIT_class = []

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives
test_sen_purpose = []
test_sen_purpose_class = []

cnt = 0
cnt0 = 0
cnt1 = 0
cnt2 = 0
cnt3 = 0
choice = 0
allFiles = allFiles[:85]

sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives

sen3_pos = 0

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head
#     print(len(list(soup.descendants)))
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
#         print(element)
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
#                 print(len(string))
                start_index.append(count)
#                 print(count)
    doc = '\n'.join(text)
    return doc


for file in allFiles:
    dictCollection = {"Action First-Party":[],
                      "Purpose":[],
                      "Personal Information Type":[],
                      "None":[]
                      }

    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    # df_tail = df.tail(1)[4]
    # print(df_tail)
    number_of_segments = len(df) + 1

    file = file.split('Test/', 1)[1]
#     print(file)

    ### beautifulsuop on html doc
    file_html = 'Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
#     print(file_html)
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)
#     print(len(indeces))
#     print(len(ls))
    ###
    
    
    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            choice = 1
            parse_json = json.loads(str(df[6][i]))
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                count_11 += 1
#                 total_data_samples0.append(parse_json["Action First-Party"]["selectedText"])
#                 total_data_number0.append(0)
#                 total_data_label0.append("Action First-Party")
                str_index = parse_json["Action First-Party"]["startIndexInSegment"]
                end_index = parse_json["Action First-Party"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_1 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sentence_list.append([ls[index],"Action_First-Party_positive","positive"])
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Action_First-Party_positive")
                        sen0_pos += 1
                        sen1_neg += 1
                        sen2_neg += 1                        
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("pos")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                    else: 
                        test_sentence_list.append([ls[index],"Action_First-Party_negative","negative"])
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1                                                
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Action_First-Party_negative")
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                cnt0 = cnt0+1
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                count_22 += 1
#                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
#                 total_data_number1.append(1)
#                 total_data_label1.append("Personal Information Type")                
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_2 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Personal_Information_Type_positive")
                        sen1_pos += 1
                        sen0_neg += 1
                        sen2_neg += 1
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("pos")                        
                        test_sen_purpose_class.append("neg")
                    else: 
                        test_sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Personal_Information_Type_negative")
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1                                                
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                cnt1 = cnt1+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                count_33 += 1                
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose")     
                str_index = parse_json["Purpose"]["startIndexInSegment"]
                end_index = parse_json["Purpose"]["endIndexInSegment"]
                for index in range(len(ls)):
                    count_3 += 1
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sentence_list.append([ls[index],"Purpose_positive","positive"])
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Purpose_positive")
                        sen0_neg += 1
                        sen1_neg += 1                        
                        sen2_pos += 1
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("pos")
                    else: 
                        test_sentence_list.append([ls[index],"Purpose_negative","negative"])
                        test_sentence_text.append(ls[index])
                        test_sentence_class.append("Purpose_negative")
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1                                                
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                cnt2 = cnt2+1
#         if df[5][i] == "Third Party Sharing/Collection":
#             parse_json = json.loads(str(df[6][i]))
# #             print(parse_json)
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
# #                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
# #                 total_data_number1.append(1)
# #                 total_data_label1.append("Personal Information Type") 
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
# #                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
# #                 total_data_number2.append(2)
# #                 total_data_label2.append("Purpose")
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_3 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sentence_list.append([ls[index],"Purpose_positive","positive"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Purpose_positive")
#                         sen2_pos += 1
#                     else: 
#                         test_sentence_list.append([ls[index],"Purpose_negative","negative"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Purpose_negative")
#                         sen2_neg += 1
#                 cnt2 = cnt2+1
#         if df[5][i] == "User Choice/Control":
# #             choice = 1
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
# #                 test_total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
# # #                 print(parse_json["Personal Information Type"]["startIndexInSegment"],parse_json["Personal Information Type"]["endIndexInSegment"])
# #                 total_data_number1.append(1)
# #                 total_data_label1.append("Personal Information Type")
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
# #                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
# #                 total_data_number2.append(2)
# #                 total_data_label2.append("Purpose")  
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_3 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sentence_list.append([ls[index],"Purpose_positive","positive"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Purpose_positive")
#                         sen2_pos += 1
#                     else: 
#                         test_sentence_list.append([ls[index],"Purpose_negative","negative"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Purpose_negative")
#                         sen2_neg += 1
#                 cnt2 = cnt2+1
#         if df[5][i] == "Data Retention":
# #             choice = 1
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
# #                 total_data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
# #                 total_data_number1.append(1)
# #                 total_data_label1.append("Personal Information Type")    
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     count_2 += 1
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_positive","positive"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_positive")
#                         sen1_pos += 1
#                     else: 
#                         test_sentence_list.append([ls[index],"Personal_Information_Type_negative","negative"])
#                         test_sentence_text.append(ls[index])
#                         test_sentence_class.append("Personal_Information_Type_negative")
#                         sen1_neg += 1
#                 cnt1 = cnt1+1
#         if choice==0:
# #             print("practis is -->", df[5][i])
#             parse_json = json.loads(str(df[6][i]))
# #             print(parse_json)
# #             print(len(df[6][i]))
# #             print(parse_json.keys())
#             attributes = parse_json.keys()
# #             print(attributes)
#             for k in attributes:
# #                 print(k)
#                 if parse_json[k]['startIndexInSegment'] != -1:
# #                     print(parse_json[k]['selectedText'])
# #                     total_data_samples3.append(parse_json[k]['selectedText'])
# #                     total_data_number3.append(3)
# #                     total_data_label3.append("None")
#                     str_index = parse_json[k]["startIndexInSegment"]
#                     end_index = parse_json[k]["endIndexInSegment"]
#                     for index in range(len(ls)):
#                         count_3 += 1
#                         if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                            (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                            (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                            (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                             test_sentence_list.append([ls[index],"None_positive","positive"])
#                             test_sentence_text.append(ls[index])
#                             test_sentence_class.append("none")
#                             sen3_pos += 1
                        
#                     cnt3 = cnt3+1

                
# print("----- check counts")                
# print(count_1, count_11)
# print(count_2, count_22)
# print(count_3, count_33)                
                
                
                
print("----- postive and -ve sentences")                
print(sen0_pos,sen0_neg)
print(sen1_pos,sen1_neg)
print(sen2_pos,sen2_neg)
print(len(test_sentence_list))


print("----- postive and -ve sentences count")                
print(len(test_sen_actioFP),len(test_sen_actioFP_class))
print(len(test_sen_personalIT),len(test_sen_personalIT_class))
print(len(test_sen_purpose),len(test_sen_purpose_class))
print(len(test_sentence_list))



# print("---")
# print(cnt0)
# print(cnt1)
# print(cnt2)
# print(cnt3)
# print("--")
# print(cnt0+cnt1+cnt2+cnt3)
# # print(len(total_data_samples0))
# # print(len(total_data_samples1))
# # print(len(total_data_samples2))
# # print(len(total_data_samples3)) # 22536

# print(len(test_sentence_text))





len of all files 25
----- postive and -ve sentences
1791 673428
2286 672933
2175 673044
675219
----- postive and -ve sentences count
675219 675219
675219 675219
675219 675219
675219


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts = count_vect.fit_transform(sen_actioFP)
X_train_counts
count_vect.get_feature_names()
count_vect.get_stop_words()

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(sen_actioFP, sen_actioFP_class)

In [39]:
import numpy as np
predicted = text_clf.predict(test_sen_actioFP)
np.mean(predicted == test_sen_actioFP_class)
#0.99706813025198082
#0.99648114167403468

0.99648114167403468

In [40]:
from sklearn import metrics
print(metrics.classification_report(test_sen_actioFP_class, predicted))

metrics.confusion_matrix(test_sen_actioFP_class, predicted)

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00    673428
        pos       0.11      0.05      0.07      1791

avg / total       1.00      1.00      1.00    675219



array([[672759,    669],
       [  1707,     84]])

In [61]:
# Produce two tables: 
# one with counts of sought text spans and 
# one with counts of sentences, labeled positive and negative, for the sought attributes.

from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head
#     print(len(list(soup.descendants)))
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
#         print(element)
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
#                 print(len(string))
                start_index.append(count)
#                 print(count)
    doc = '\n'.join(text)
    return doc

for file in allFiles:
    ls = []
    indeces = []
    data = html_to_text(file)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
#         print(count)
#         print(l)
        count = count+len(l)
print(indeces)

[[0, 83], [83, 179], [179, 282], [282, 827], [827, 934], [934, 1064], [1064, 1176], [1176, 1230], [1230, 1427], [1427, 1652], [1652, 1750], [1750, 1883], [1883, 2000], [2000, 2104], [2104, 2235], [2235, 2300], [2300, 2373], [2373, 2501], [2501, 2591], [2591, 2710], [2710, 2853], [2853, 3011], [3011, 3166], [3166, 3298]]


In [41]:
set(test_sentence_class)

{'Action_First-Party_negative',
 'Action_First-Party_positive',
 'Personal_Information_Type_negative',
 'Personal_Information_Type_positive',
 'Purpose_negative',
 'Purpose_positive',
 'none'}

In [42]:
set(sentence_class)

{'Action_First-Party_negative',
 'Action_First-Party_positive',
 'Personal_Information_Type_negative',
 'Personal_Information_Type_positive',
 'Purpose_negative',
 'Purpose_positive',
 'none'}